# Simple LLM Application

[Tutorial](https://python.langchain.com/docs/tutorials/llm_chain/)

## Setup

In [3]:
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

assert os.environ["LANGSMITH_TRACING"] is not None
assert os.environ["LANGSMITH_API_KEY"] is not None
assert os.environ["LANGSMITH_PROJECT"] is not None
assert os.environ["OPENAI_API_KEY"] is not None

In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

## ChatModels

- ChatModels are instances of LangChain [Runnables](https://python.langchain.com/docs/concepts/runnables/), which means they expose a standard interface for interacting with them.
- ChatModels receive [message](https://python.langchain.com/docs/concepts/messages/) objects as input and generate message objects as output.
- In addition to text content, message objects convey conversational [roles](https://python.langchain.com/docs/concepts/messages/#role) and hold important data, such as tool calls and token usage counts.

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Arabic"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='مرحبًا!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 20, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BtqXhoXU5mMCcVHuOB8zeQnW2EmuY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d160c7bb-f7eb-4158-80f9-c2168c6ad727-0', usage_metadata={'input_tokens': 20, 'output_tokens': 4, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

LangChain also supports chat model inputs via strings or OpenAI format. The following are equivalent:

```python
model.invoke("Hello")
model.invoke([{"role": "user", "content": "Hello"}])
model.invoke([HumanMessage("Hello")])
```

Runnables such as ChatModels expose a standard interface which includes streaming responses.

In [7]:
for token in model.stream(messages):
    print(token.content, end="|")

|مر|حب|ًا|!||

## Prompt Templates

- Prompt Templates are like prepared statements in SQL.
- [Prompt templates](https://python.langchain.com/docs/concepts/prompt_templates/) are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

In [15]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

## Prompt Template composes the prompt using input
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")])
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [16]:
model.invoke(prompt)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BtqksEqe6NiQcCW7CUxNa9j3dSWi3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4ca0e3ab-9ac9-4222-bfa1-1d03cb7d011f-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})